<a href="https://colab.research.google.com/github/haoruilee/js_competitions/blob/kaggle-upload-1/js-transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
import polars as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
import os
import statistics as stat
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, mean_squared_error
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
!ls drive/MyDrive/jane-street-real-time-market-data-forecasting

features.csv	   lags.parquet    sample_submission.csv  train.parquet
kaggle_evaluation  responders.csv  test.parquet


In [5]:
!ls ./drive/jane-street-real-time-market-data-forecasting

ls: cannot access './drive/jane-street-real-time-market-data-forecasting': No such file or directory


In [8]:

jane_street_real_time_market_data_forecasting_path = "./drive/MyDrive/jane-street-real-time-market-data-forecasting"  # Set your path here
checkpoint_dir = "./drive/MyDrive/kaggle_js_checkpoints_transformer"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import polars as pl
import numpy as np
import os
import pandas as pd

# Load data using Parquet format
jane_street_real_time_market_data_forecasting_path = "./drive/MyDrive/jane-street-real-time-market-data-forecasting"  # Set your path here

valid_from = 100  # Example valid_from date threshold

# Load data
alltraindata = pl.scan_parquet(f"{jane_street_real_time_market_data_forecasting_path}/train.parquet")
train = alltraindata.filter(pl.col("date_id") >= valid_from).collect()

# Prepare features
feature_names = [f"feature_{i:02d}" for i in range(79)]
train_features = train.select(feature_names)
train_features = train_features.fill_null(strategy='forward').fill_null(0)

# Prepare data
X = train_features.to_numpy()
y = train.select([col for col in train.columns if col.startswith('responder_')]).to_numpy()  # Use all responder columns

# Define sequence length
seq_len = 10  # Number of time steps in each sequence

# Create sequences
X_seq = []
y_seq = []

for i in range(len(X) - seq_len):
    X_seq.append(X[i:i + seq_len])  # Create a sequence of length `seq_len`
    y_seq.append(y[i + seq_len])  # Predict the next point after the sequence

# Convert lists to numpy arrays
X_seq = np.array(X_seq)  # Shape: (num_sequences, seq_len, input_size)
y_seq = np.array(y_seq)  # Shape: (num_sequences, output_size)

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X_seq, dtype=torch.float32).cuda()  # Move data to GPU
y_tensor = torch.tensor(y_seq, dtype=torch.float32).cuda()  # Move data to GPU

# Create DataLoader for batching
dataset = TensorDataset(X_tensor, y_tensor)
batch_size = 2
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Transformer Model Definition
class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model=128, nhead=4, num_layers=2, dropout=0.2, output_size=6):
        super(TransformerModel, self).__init__()
        self.input_projection = nn.Linear(input_size, d_model)
        self.pos_encoder = nn.Sequential(
            nn.Dropout(dropout)
        )
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )
        self.output_layer = nn.Linear(d_model, output_size)  # Output matches number of responder columns

    def forward(self, x):
        x = self.input_projection(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.output_layer(x[:, -1, :])  # Use the output from the last time step
        return x.squeeze()

# Instantiate the model
input_size = X_seq.shape[2]  # Number of features per time step
output_size = y_seq.shape[1]  # Number of responder columns
model = TransformerModel(input_size=input_size, d_model=128, nhead=4, num_layers=2, dropout=0.2, output_size=output_size).cuda()  # Move model to GPU

# Loss and optimizer
criterion = torch.nn.MSELoss()  # Base loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Custom weighted loss function to prioritize responder_6
def weighted_loss(outputs, targets):
    loss = F.mse_loss(outputs, targets, reduction='none')

    # Apply a higher weight for the rows corresponding to responder_6
    weights = torch.ones_like(targets).cuda()  # Ensure weights are on GPU
    responder_6_indices = (updated_responders_df['responder'] == 'responder_6').values.nonzero()[0]
    for idx in responder_6_indices:
        weights[idx, :] = 5.0  # Assign higher weight to responder_6

    weighted_loss = loss * weights
    return weighted_loss.mean()

# Training loop
checkpoint_dir = "./drive/MyDrive/kaggle_js_checkpoints_transformer"
os.makedirs(checkpoint_dir, exist_ok=True)
epochs = 50
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    for batch_X, batch_y in data_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X)

        # Compute weighted loss
        loss = weighted_loss(outputs, batch_y)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Print loss for every epoch
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

    # Save checkpoint after every epoch
    checkpoint_path = os.path.join(checkpoint_dir, f"transformer_checkpoint_epoch_{epoch+1}.pth")
    torch.save(model.state_dict(), checkpoint_path)
    print(f"Checkpoint saved as {checkpoint_path}")

# Save the final trained model
torch.save(model.state_dict(), "transformer_model.pth")
print("Model saved as transformer_model.pth")


In [ ]:
if run_type == 'Transformer':
    # 模型参数
    input_size = 79
    d_model = 512
    nhead = 8
    num_layers = 3
    dropout = 0.1

    # 初始化模型
    model = TransformerModel(
        input_size=input_size,
        d_model=d_model,
        nhead=nhead,
        num_layers=num_layers,
        dropout=dropout
    ).to(device)

    print_model_size(model)

    # 优化器设置
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
    loss_function = nn.MSELoss(reduction='none')

    # 训练循环
    epochs = 20
    best = float('-inf')
    degraded = 0
    best_model = model

    for epoch in range(epochs):
        train_loss, train_mse, train_r2 = train_model(
            model, train_loader, optimizer, loss_function, device
        )

        scheduler.step()
        val_mse, val_r2 = evaluate_model(model, val_loader)

        print(f'epoch {epoch}:')
        print(f'train loss {train_loss:.4f}, train_r2 {train_r2:.4f}, '
              f'train_mse {train_mse:.4f}')
        print(f'val_mse {val_mse:.4f}, val_r2 {val_r2:.4f}')
        print(f'lr: {scheduler.get_last_lr()[0]:.6f}')

        if val_r2 > best:
            best = val_r2
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), f'./model/js_{run_type}_unnorm.pth')
            degraded = 0
        else:
            degraded += 1

        if degraded > 10:
            print("Early stopping triggered")
            break

        torch.cuda.empty_cache()

    model = best_model
    test_mse, test_r2 = evaluate_model(model, test_loader)
    print(f'test R2 score is {test_r2}')


In [ ]:
lags_ : pl.DataFrame | None = None

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make predictions using either LSTM or Transformer model"""
    global lags_
    if lags is not None:
        lags_ = lags

    # 初始化预测DataFrame
    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )

    # 选择特征列
    feature_cols = [f"feature_{i:02d}" for i in range(79)]

    # 处理测试数据
    test_features = test.select(feature_cols)
    test_features = test_features.fill_null(strategy='forward').fill_null(0)

    # 转换为tensor
    X_test = test_features.to_numpy()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

    # 进行预测
    model.eval()
    with torch.no_grad():
        pred = model(X_test_tensor).cpu().numpy()

    # 更新预测结果
    predictions = predictions.with_columns(pl.Series('responder_6', pred.ravel()))

    # 验证输出格式
    assert isinstance(predictions, (pl.DataFrame, pd.DataFrame))
    assert list(predictions.columns) == ['row_id', 'responder_6']
    assert len(predictions) == len(test)
    print(predictions)
    return predictions

In [ ]:
import os

import pandas as pd
import polars as pl

import kaggle_evaluation.jane_street_inference_server


inference_server = kaggle_evaluation \
                        .jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )
